In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
votes_json = pd.read_json('../data/output/plenary/json/leg-55/votes.json', dtype=False)
votes_json

,voting_id,vote_type,politician_id
0,55_004_v1,YES,7346
1,55_004_v1,YES,7425
2,55_004_v1,YES,6873
3,55_004_v1,YES,913
4,55_004_v1,YES,1080
...,...,...,...
402390,55_309_v92,NO,7270
402391,55_309_v92,NO,4371
402392,55_309_v92,NO,7102
402393,55_309_v92,NO,6701


In [4]:
politicians_json = pd.read_json('../data/output/politician/leg-55/politicians.json', dtype=False)
politicians_json

,id,full_name,party
0,7220,Moyaers Bert,Vooruit
1,7261,Goblet Marc,PS
2,6759,Hedebouw Raoul,PVDA-PTB
3,6309,Hennuy Laurence,Ecolo-Groen
4,7076,Samyn Ellen,VB
...,...,...,...
188,6326,Özen Özlem,PS
189,7022,Freilich Michael,N-VA
190,7171,Tison Philippe,PS
191,7034,Leysen Christian,Open Vld


In [5]:
joined = pd.merge(votes_json, politicians_json, left_on='politician_id', right_on='id', how='left')
joined

,voting_id,vote_type,politician_id,id,full_name,party
0,55_004_v1,YES,7346,7346,Briers Jan,cd&v
1,55_004_v1,YES,7425,7425,Bury Katleen,VB
2,55_004_v1,YES,6873,6873,Creyelman Steven,VB
3,55_004_v1,YES,913,913,Dedecker Jean-Marie,ONAFH
4,55_004_v1,YES,1080,1080,Depoortere Ortwin,VB
...,...,...,...,...,...,...
402390,55_309_v92,NO,7270,7270,Moscufo Nadia,PVDA-PTB
402391,55_309_v92,NO,4371,4371,Prévot Maxime,Les Engagés
402392,55_309_v92,NO,7102,7102,Rohonyi Sophie,DéFI
402393,55_309_v92,NO,6701,6701,Van Hees Marco,PVDA-PTB


In [6]:
df = joined[['voting_id', 'vote_type', 'politician_id', 'party']]
df

,voting_id,vote_type,politician_id,party
0,55_004_v1,YES,7346,cd&v
1,55_004_v1,YES,7425,VB
2,55_004_v1,YES,6873,VB
3,55_004_v1,YES,913,ONAFH
4,55_004_v1,YES,1080,VB
...,...,...,...,...
402390,55_309_v92,NO,7270,PVDA-PTB
402391,55_309_v92,NO,4371,Les Engagés
402392,55_309_v92,NO,7102,DéFI
402393,55_309_v92,NO,6701,PVDA-PTB


In [7]:
# count votes by voting id an party
vote_counts = df.groupby(['voting_id', 'party', 'vote_type']).size().unstack(fill_value=0)
vote_counts

vote_type               ABSTENTION  NO  YES
voting_id  party                           
55_004_v1  DéFI                  0   2    0
           Ecolo-Groen           0  20    0
           INDEP                 0   1    0
           Les Engagés           0   5    0
           MR                    0  12    0
...                            ...  ..  ...
55_309_v92 PVDA-PTB              1   9    0
           VB                    0   0   18
           Vooruit               0   0    7
           cd&v                  0   0   12
           sp.a                  0   0    1

[39105 rows x 3 columns]

# Stemmingen waar binnen een partij zowel ja als nee gestemd is

In [18]:
inconsistent_votes = vote_counts[(vote_counts > 0).sum(axis=1) >= 2]
party_disagreements = inconsistent_votes[inconsistent_votes['ABSTENTION'] == 0 ]
party_disagreements

,vote_type,ABSTENTION,NO,YES
voting_id,party,,,
55_004_v1,cd&v,0,10,1
55_004_v13,cd&v,0,6,5
55_004_v5,cd&v,0,10,1
55_017_v4,PVDA-PTB,0,1,10
55_017_v6,MR,0,9,1
...,...,...,...,...
55_304_v36,PS,0,1,13
55_304_v63,VB,0,1,15
55_309_v12,cd&v,0,11,1


# Aantal stemmingen waar er zowel 'ja' als 'nee' gestemd is per partij

In [19]:
party_disagreements.index.get_level_values(1).value_counts()

party
N-VA           70
PS             49
cd&v           46
Les Engagés    39
DéFI           30
Ecolo-Groen    30
MR             28
PVDA-PTB       24
Open Vld       21
VB             20
Vooruit         4
Name: count, dtype: int64